In [1]:
!pip3 install sentencepiece
!pip3 install tf-sentencepiece
#!pip install tensorflow==1.13.1

     |████████████████████████████████| 1.0MB 3.5MB/s 
     |████████████████████████████████| 2.4MB 3.5MB/s 


In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
gitDir = "/content/gdrive/My Drive/nlp/"
os.chdir(gitDir + "data/")
print(os.listdir("."))

#sys.path.insert(0,gitDir + "data")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
['Kaggle Amazon Fine Food.ipynb', '.ipynb_checkpoints', 'Organic Dataset.ipynb', 'cache', '__pycache__', 'Amazon Reviews.ipynb', 'Amazon Multilingual.ipynb', 'tmpBACKUP.py', 'tmp.ipynb', 'amazon_multilingual.py', 'Dataset_Amazon_Multilingual.ipynb']


In [8]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

import amazon_multilingual
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

1.13.1
Version:  1.13.1
Eager mode:  False
Hub version:  0.4.0
GPU is available


## create graph

In [0]:
model = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling-many/1", trainable=False)

In [0]:
batchSize = 128

In [12]:
iterator_train, feed_dict_train, length_train = amazon_multilingual.getData("UK", shuffle=True, buffer=batchSize, batchsize=batchSize)

cache/amazon_multilingual_UK_category.npy and cache/amazon_multilingual_UK_labels.npy  exist. Using saved npy files...


In [13]:
iterator_test, feed_dict_test, length_test = amazon_multilingual.getData("DE", shuffle=False, buffer=batchSize, batchsize=batchSize)

cache/amazon_multilingual_DE_category.npy and cache/amazon_multilingual_DE_labels.npy  exist. Using saved npy files...
dataset is not shuffled and prefetched


In [31]:
classes = 35

#text_input = tf.placeholder(dtype=tf.string, shape=[None])
text_input, label = iterator_train.get_next()
embedded_text = model(text_input)

output = tf.layers.dense(embedded_text, classes)
#output = tf.keras.layers.Dense(classes, activation='relu')(embedded_text)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(label, classes), logits=output))
train_op = tf.train.AdamOptimizer().minimize(loss)

#acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(tf.one_hot(label, classes), 1), predictions=tf.argmax(output,1))
preds = tf.argmax(output,1)
acc, acc_op = tf.metrics.accuracy(labels=label, predictions=preds )


init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer(),tf.local_variables_initializer()])
session = tf.Session()
session.run(init_op)
session.run(iterator_train.initializer, feed_dict=feed_dict_train)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0514 19:17:25.402078 140253607696256 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
#_, loss_value = session.run([train_op, loss], feed_dict={text_input: ["Puppies are nice."], label: [1] })
#print(loss_value)

#print(session.run([acc]))

#prediction = session.run(prediction, feed_dict={text_input: ["Puppies are nice."], label: [1] })
#print(type(prediction), prediction)

[0.0]


# train

In [16]:
stepsPerEpoch = int(length_train / batchSize)
epochs = 1
steps = stepsPerEpoch * epochs
print(steps)

13326


In [32]:
%%time
for i in range(15000):
  _, loss_value = session.run([train_op, loss])
  if i%1000 == 0:
    print(i, loss_value)
  #print(loss_value)

0 3.5767918
1000 0.9267002
2000 0.77375996
3000 0.6920674
4000 0.8503573
5000 0.60737556
6000 0.6321069
7000 0.7100878
8000 0.74716985
9000 0.70631367
10000 0.6300449
11000 0.7574476
12000 0.79972374
13000 0.7070496


OutOfRangeError: ignored